# Point to Surface distance

In [1]:
import trimesh
import numpy as np
from scipy.spatial import cKDTree

# Load meshes
gt_mesh = trimesh.load('../assets/pifuhd_results/result_female-1-casual-frame_0016_real_B_512.obj')
pred_mesh = trimesh.load('../assets/pifuhd_results/result_female-1-casual-frame_0016_fake_B_512.obj')

# Sample points from predicted mesh
pred_points = pred_mesh.sample(5000)

# Build a KD-tree for the ground-truth mesh surface
gt_points = gt_mesh.sample(10000)
gt_tree = cKDTree(gt_points)

# Find closest surface point in GT for each predicted point
distances, _ = gt_tree.query(pred_points)
p2s_distance = np.mean(distances)
print("Point-to-Surface (P2S) Distance:", p2s_distance)

Point-to-Surface (P2S) Distance: 0.04563946491437865


In [6]:
import trimesh
import numpy as np
from scipy.spatial import cKDTree
from sklearn.metrics import auc

# Load meshes
def load_mesh(file_path):
    return trimesh.load_mesh(file_path)

# Extract point cloud from mesh (sampling points on the surface)
def get_point_cloud(mesh, num_points=10000):
    return mesh.sample(num_points)

# Compute P2S distances between two point clouds
def compute_p2s_distances(gt_points, pred_points):
    gt_tree = cKDTree(gt_points)
    distances, _ = gt_tree.query(pred_points)
    return distances

# Compute the CED and AUC
def compute_ced_auc(distances, thresholds=np.linspace(0, 0.05, 100)):
    cumulative = np.array([(distances < t).sum() / len(distances) for t in thresholds])
    return cumulative

def ced_at_threshold(distances, threshold=0.01):
    return np.mean(distances < threshold)

# Load your ground truth and reconstructed meshes
gt_mesh = trimesh.load('../assets/pifuhd_results/result_female-1-casual-frame_0016_real_B_512.obj')
pred_mesh = trimesh.load('../assets/pifuhd_results/result_female-1-casual-frame_0016_fake_B_512.obj')

# Extract point clouds (we'll sample 10000 points for this example)
gt_points = get_point_cloud(gt_mesh, num_points=10000)
pred_points = get_point_cloud(pred_mesh, num_points=10000)

# Compute P2S distances
distances = compute_p2s_distances(gt_points, pred_points)

# Compute CED curve
ced_curve = compute_ced_auc(distances)

# Optionally, compute AUC (Area Under CED Curve)
ced_auc = auc(np.linspace(0, 0.05, 100), ced_curve)
print(f"AUC-CED: {ced_auc}")

# Calculate CED @ 1cm
recall_at_1cm = ced_at_threshold(distances, 0.01)
print(f"CED @ 1cm: {recall_at_1cm}")


AUC-CED: 0.01898997474747475
CED @ 1cm: 0.0798


In [12]:
import trimesh
import numpy as np
from scipy.spatial import cKDTree

# Load meshes
def load_mesh(file_path):
    return trimesh.load_mesh(file_path)

# Extract point cloud from mesh (sampling points on the surface)
def get_point_cloud(mesh, num_points=10000):
    return mesh.sample(num_points)

# Compute Chamfer Distance between two point clouds
def chamfer_distance(gt_points, pred_points):
    # Build KDTree for both point clouds
    gt_tree = cKDTree(gt_points)
    pred_tree = cKDTree(pred_points)
    
    # Find closest point in the ground truth for each predicted point
    dist_pred_to_gt, _ = pred_tree.query(gt_points)
    
    # Find closest point in the predicted for each ground truth point
    dist_gt_to_pred, _ = gt_tree.query(pred_points)
    
    # Compute Chamfer distance
    chamfer_dist = np.mean(dist_pred_to_gt) + np.mean(dist_gt_to_pred)
    return chamfer_dist

# Load your ground truth and reconstructed meshes
gt_mesh = trimesh.load('../assets/pifuhd_results/result_female-1-casual-frame_0016_real_B_512.obj')
pred_mesh = trimesh.load('../assets/pifuhd_results/result_female-1-casual-frame_0016_fake_B_512.obj')

# Extract point clouds (we'll sample 10000 points for this example)
gt_points = get_point_cloud(gt_mesh, num_points=1000)
pred_points = get_point_cloud(pred_mesh, num_points=1000)

# Compute Chamfer distance
chamfer_dist = chamfer_distance(gt_points, pred_points)
print(f"Chamfer Distance: {chamfer_dist}")


Chamfer Distance: 0.380847760338351
